In [95]:
#Iván Andrés Trujillo Abella 
#ivantrujillo1229@gmail.com
#Consolidate Thesis

In [96]:
import os
os.chdir(r"C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets")

In [97]:
## Own functions
def error_in_date(x, format_date= "%Y-%m-%d"):
    try:
        datetime.strptime(x,format_date)
        return False
    except:
        return True

def clean_date(x):
    return pd.to_datetime(x[0:10], format= "%Y-%m-%d")    
    
def cleantime(x):
    if type(x)==str:
        return x.replace("00:00:00", "").replace(" ", "")
    else:
        return x

def top_cat(k, cat, df):
    otro = []
    cat_relative_dict =  df[cat].value_counts(normalize=True).to_dict()
    for cat in cat_relative_dict:
        if cat_relative_dict[cat]<=k:
            otro.append(cat)
    return otro


def bankyear(df_):
    df = df_.copy()
    years = [2016, 2017, 2018, 2019]
    df['bankrupt-year'] = 0
    for firm in df.index:
        unchanged = True
        for year in years:
            if df.loc[firm, 'evento'+str(year)]==1 and unchanged==True:
                df.loc[firm,'bankrupt-year']= year
                unchanged = False
    return df['bankrupt-year']



def new_firms(current_df, next_df, key):
    firm_first =  set(current_df[key].unique()) -  set(next_df[key].unique())
    firm_second =  set(next_df[key].unique()) -  set(current_df[key].unique())
    return pd.concat([current_df[current_df[key].isin(firm_first)] , next_df[next_df[key].isin(firm_second)]])



def check_vars(vars_to_check, df):
    not_in = []
    for var in vars_to_check:
        if var not in df.columns:
            not_in.append(var)  
    return not_in


In [98]:
import pandas as pd
import numpy as np

In [99]:
# Loading datasets
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
      dfs['df'+str(year)]  = pd.read_excel('Carátula-Pymes-'+str(year)+'.xlsx')

        
"""
## To flowchart
for year in years:
    print('#'*10)
    print(dfs['df'+str(year)].shape , year)
    print(dfs['df'+str(year)]['Estado actual'].value_counts())
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Estado actual']!='EN ETAPA PREOPERATIVA']
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['Fecha'] = pd.to_datetime( dfs['df'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.month==12] # there is duplicates given present the information two or more times..
    print(dfs['df'+str(year)].shape, year)
    dfs['df'+str(year)]['evento'+str(year)]  = np.where(dfs['df'+str(year)]['Estado actual']!='ACTIVA',1 ,0)
    print('--'*100)
    print(dfs['df'+str(year)]['evento'+str(year)].value_counts())
"""

'\n## To flowchart\nfor year in years:\n    print(\'#\'*10)\n    print(dfs[\'df\'+str(year)].shape , year)\n    print(dfs[\'df\'+str(year)][\'Estado actual\'].value_counts())\n    dfs[\'df\'+str(year)]  = dfs[\'df\'+str(year)][dfs[\'df\'+str(year)][\'Estado actual\']!=\'EN ETAPA PREOPERATIVA\']\n    print(dfs[\'df\'+str(year)].shape , year)\n    dfs[\'df\'+str(year)][\'Fecha\'] = pd.to_datetime( dfs[\'df\'+str(year)][ \'Fecha de Corte\'], format= "%Y-%m-%d")\n    dfs[\'df\'+str(year)]  = dfs[\'df\'+str(year)][dfs[\'df\'+str(year)][\'Fecha\'].dt.month==12] # there is duplicates given present the information two or more times..\n    print(dfs[\'df\'+str(year)].shape, year)\n    dfs[\'df\'+str(year)][\'evento\'+str(year)]  = np.where(dfs[\'df\'+str(year)][\'Estado actual\']!=\'ACTIVA\',1 ,0)\n    print(\'--\'*100)\n    print(dfs[\'df\'+str(year)][\'evento\'+str(year)].value_counts())\n'

In [100]:
for year in years:
    print('#'*10)
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Estado actual']!='EN ETAPA PREOPERATIVA']
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['Fecha'] = pd.to_datetime( dfs['df'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.month==12] # there is duplicates given present the information two or more times..
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.day==31] # there is duplicates given present the information two or more times..
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)] =  dfs['df'+str(year)][-pd.isnull(dfs['df'+str(year)]['NIT'])]
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['evento'+str(year)]  = np.where(dfs['df'+str(year)]['Estado actual']!='ACTIVA',1 ,0)
    dfs['df'+str(year)] = dfs['df'+str(year)][['evento'+str(year), 'NIT' ,'Clasificación Industrial Internacional Uniforme Versión 4 A.C']]
    print(dfs['df'+str(year)]['NIT'].duplicated().sum())
    print(dfs['df'+str(year)].shape , year)
# There is a only repeated firm in 2017 but is a error..

##########
(17799, 31) 2016
(17723, 31) 2016
(17701, 32) 2016
(17701, 32) 2016
0
(17701, 3) 2016
##########
(14696, 32) 2017
(14656, 32) 2017
(14639, 33) 2017
(14638, 33) 2017
0
(14638, 3) 2017
##########
(14275, 34) 2018
(14229, 34) 2018
(14211, 35) 2018
(14209, 35) 2018
0
(14209, 3) 2018
##########
(19185, 36) 2019
(19140, 36) 2019
(19128, 37) 2019
(19125, 37) 2019
0
(19125, 3) 2019


In [101]:
dfs['df2018']['NIT'].duplicated().sum()

0

In [102]:
dfs['df2019']['evento2019'].value_counts() # Here there are 876 firms,  but since declarate the process.

evento2019
0    18249
1      876
Name: count, dtype: int64

In [103]:
for year in range(2016,2020):
    print(dfs['df'+str(year)]['evento'+str(year)].value_counts())

evento2016
0    17193
1      508
Name: count, dtype: int64
evento2017
0    14019
1      619
Name: count, dtype: int64
evento2018
0    13444
1      765
Name: count, dtype: int64
evento2019
0    18249
1      876
Name: count, dtype: int64


In [104]:
full = dfs['df2016'].copy()
for year in range(2017,2020):
    clients= new_firms(full,dfs['df'+str(year)], 'NIT')
    full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)
    full = pd.concat([full, clients])

In [105]:
full['NIT'].duplicated().sum()

0

In [106]:
full[full['NIT'].duplicated(keep = False)].to_excel('what.xlsx')

In [107]:
full['evento2019'].value_counts()

evento2019
0.0    18249
1.0      876
Name: count, dtype: int64

In [108]:
"""
H = pd.DataFrame( [[1,2,3,4,5], [np.nan, np.nan, np.nan, np.nan, np.nan],
                   [np.nan, np.nan, np.nan, np.nan, 1]  ,  [0, np.nan, np.nan, np.nan, np.nan] ])
H.sum(axis=1, numeric_only=True, min_count=1) # This is very important... 
"""

'\nH = pd.DataFrame( [[1,2,3,4,5], [np.nan, np.nan, np.nan, np.nan, np.nan],\n                   [np.nan, np.nan, np.nan, np.nan, 1]  ,  [0, np.nan, np.nan, np.nan, np.nan] ])\nH.sum(axis=1, numeric_only=True, min_count=1) # This is very important... \n'

In [109]:
full[['evento2016', 'evento2017', 'evento2018', 'evento2019']].sum(axis=1, min_count=1).value_counts(dropna=False)

0.0    25931
1.0      508
2.0      300
4.0      247
3.0      224
Name: count, dtype: int64

In [110]:
full['suma'] = full[['evento2016', 'evento2017', 'evento2018', 'evento2019']].sum(axis=1, min_count=1)

In [111]:
full['indicator'] = (full['suma']!=0).astype('int')

In [112]:
full['indicator'].value_counts()

indicator
0    25931
1     1279
Name: count, dtype: int64

In [113]:
full['NIT'].duplicated().sum()

0

In [114]:
full.shape

(27210, 8)

In [115]:
full = full[-(full['NIT'].duplicated())].reset_index(drop=True)
print(full.shape)
full['time-event'] =bankyear(full)
full['time-event'].value_counts(dropna=False)

(27210, 8)


time-event
0       25931
2016      508
2019      282
2018      257
2017      232
Name: count, dtype: int64

In [116]:
full['indicator'].value_counts()

indicator
0    25931
1     1279
Name: count, dtype: int64

In [117]:
## Draw firmst that enter to process in 2016 to avoid bias in indicators...
full = full[full['evento2016']!=1]
print(full['time-event'].value_counts())

time-event
0       25931
2019      282
2018      257
2017      232
Name: count, dtype: int64


In [118]:
full['indicator'].value_counts()

indicator
0    25931
1      771
Name: count, dtype: int64

In [119]:
full['time-event'].value_counts().sum()

26702

In [120]:
print(282+ 257 + 232)

771


In [121]:
full['event'] = (full['time-event']!=0).astype('int')

In [122]:
full['event'].value_counts()

event
0    25931
1      771
Name: count, dtype: int64

In [123]:
ERIS={}
EFSS={}
for year in range(2016,2019):
        #loops in ERI
    ERIS['eri'+str(year)] = pd.read_excel('ERI-Pymes-'+str(year)+'.xlsx')
    ERIS['eri'+str(year)]['Fecha'] = pd.to_datetime( ERIS['eri'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    ERIS['eri'+str(year)] = ERIS['eri'+str(year)][ERIS['eri'+str(year)]['Fecha'].dt.month==12]
    if year==2016:
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.year==2016]
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.month==12]
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.day==31]
    else:
        ERIS['eri'+str(year)] = ERIS['eri'+str(year)][ERIS['eri'+str(year)]['Periodo']=='Periodo Actual']
        ERIS['eri'+str(year)] = ERIS['eri'+str(year)][pd.to_datetime(ERIS['eri'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.day==31]

    #ERIS['eri'+str(year)][-ERIS['eri'+str(year)]['NIT'].duplicated()].reset_index(drop=True)

    # loops in efs
    EFSS['efs'+str(year)] = pd.read_excel('ESF-Pymes-'+str(year)+'.xlsx')
    if year == 2016:
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.year==2016]
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.month==12]
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.day==31]
    else:
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][EFSS['efs'+str(year)]['Periodo']=='Periodo Actual']
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][pd.to_datetime(EFSS['efs'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.month==12]
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][pd.to_datetime(EFSS['efs'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.day==31]

## Lines to debug...
#EFSS['efs2018']['NIT'].duplicated(keep=False).sum()
#EFSS['efs2018'][EFSS['efs2018']['NIT'].duplicated(keep=False)].to_excel('2018_review.xlsx')

In [124]:
# We can integrate both statements...

FULLS = {}
for year in range(2016,2019):
    # Number of fimrs by year in each statement!
    print('--'*10)
    print(year,EFSS['efs'+str(year)].shape[0] ,  ERIS['eri'+str(year)].shape[0])
    FULLS['full'+str(year)] = EFSS['efs'+str(year)].merge(ERIS['eri'+str(year)], on='NIT', how='outer',
                                                          indicator = True, suffixes=(None, 'to_remove') )
    duplicated = [var for var in FULLS['full'+str(year)].columns if 'to_remove' in var]
    FULLS['full'+str(year)].drop(columns=duplicated, inplace=True)
    print('--'*10)
    print(FULLS['full'+str(year)]['_merge'].value_counts(dropna=False))

--------------------
2016 17777 17777
--------------------
_merge
both          17777
left_only         0
right_only        0
Name: count, dtype: int64
--------------------
2017 14679 14679
--------------------
_merge
both          14679
left_only         0
right_only        0
Name: count, dtype: int64
--------------------
2018 14257 14257
--------------------
_merge
both          14257
left_only         0
right_only        0
Name: count, dtype: int64


In [125]:
#FULLS['full2018'].columns

In [126]:
raw_X = ['NIT','Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 
         'Patrimonio total','Total pasivos', 'Total de activos', 'Ganancias acumuladas',  
         'Otros pasivos financieros corrientes','Pasivos corrientes totales', 'Costos financieros', 
         'Otros activos no financieros corrientes', 'Activos corrientes totales']

In [127]:
# Objective : rectificy if the variable exist in all datasets with the name..
# must be appear only empty lists
for year in range(2016,2019):
    print(check_vars(raw_X, FULLS['full'+str(year)]))
    FULLS['full'+str(year)] = FULLS['full'+str(year)][raw_X]
    FULLS['full'+str(year)].columns = ['NIT'] + [col+'-'+str(year) for col in  FULLS['full'+str(year)].columns if col!='NIT']  

[]
[]
[]


In [128]:
full = full.copy()
for year in range(2016,2019):
    full = full.merge(FULLS['full'+str(year)], on='NIT', how='outer', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)

In [129]:
print(full[full['event'].isnull()].shape)
a = set(full[full['event'].isnull()]['NIT'].unique())
b = set(dfs['df2016']['NIT'].unique())
c = list(b & a)
print(len(c)) ## 

(571, 49)
508


In [130]:
full[full['event'].isnull()]

,evento2016,NIT,Clasificación Industrial Internacional Uniforme Versión 4 A.C,evento2017,evento2018,evento2019,suma,indicator,time-event,event,...,Costo de ventas-2018,Patrimonio total-2018,Total pasivos-2018,Total de activos-2018,Ganancias acumuladas-2018,Otros pasivos financieros corrientes-2018,Pasivos corrientes totales-2018,Costos financieros-2018,Otros activos no financieros corrientes-2018,Activos corrientes totales-2018
26702,NaN,800001792.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3071136.0,11177209.0,13324728.0,24501937.0,1824517.0,NaN,11510877.0,488706.0,0.0,2137895.0
26703,NaN,800002448.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,709261.0,591544.0,3297138.0,3888682.0,490107.0,NaN,35935.0,9242.0,NaN,284380.0
26704,NaN,800002819.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26705,NaN,800039321.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26706,NaN,800041672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8021553.0,538428.0,7241940.0,7780368.0,-220199.0,NaN,2305667.0,149714.0,NaN,5249441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27268,NaN,900510770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88315.0,521145.0,612373.0,1133518.0,-906194.0,NaN,62131.0,605.0,NaN,5336.0
27269,NaN,900637247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,26891.0,548.0,27439.0,-1109.0,NaN,548.0,0.0,567.0,27439.0
27270,NaN,900713252.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,838816.0,50542875.0,51381691.0,-661184.0,NaN,4202441.0,0.0,NaN,256995.0
27271,NaN,901206570.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,188917.0,-24592.0,70804.0,46212.0,-64592.0,NaN,70804.0,524.0,NaN,27614.0


In [131]:
full['event'].value_counts()

event
0.0    25931
1.0      771
Name: count, dtype: int64

In [132]:
# This cell is to check that all missing events are of 'etapa preoperativa' to avoid biases
NULAS  =full[full['event'].isnull()]['NIT'].unique()
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
    dfs['df'+str(year)]  = pd.read_excel('Carátula-Pymes-'+str(year)+'.xlsx')
    print(dfs['df'+str(year)][dfs['df'+str(year)]['NIT'].isin(NULAS)]['Estado actual'].value_counts())

Estado actual
ACUERDO DE REORGANIZACION      343
ACUERDO DE REESTRUCTURACIÓN    152
EN ETAPA PREOPERATIVA           49
CONCORDATO EN EJECUCIÓN          9
CONCORDATO EN TRÁMITE            4
Name: count, dtype: int64
Estado actual
ACUERDO DE REORGANIZACION      262
ACUERDO DE REESTRUCTURACIÓN    116
ACTIVA                          57
EN ETAPA PREOPERATIVA           29
CONCORDATO EN EJECUCIÓN          9
CONCORDATO EN TRÁMITE            1
Name: count, dtype: int64
Estado actual
ACUERDO DE REORGANIZACION      239
ACUERDO DE REESTRUCTURACIÓN     93
ACTIVA                          74
EN ETAPA PREOPERATIVA           29
CONCORDATO EN EJECUCIÓN          6
CONCORDATO EN TRÁMITE            1
Name: count, dtype: int64
Estado actual
ACUERDO DE REORGANIZACION      207
ACUERDO DE REESTRUCTURACIÓN     77
ACTIVA                          74
EN ETAPA PREOPERATIVA           20
CONCORDATO EN EJECUCIÓN          6
Name: count, dtype: int64


In [133]:
full = full[-full['event'].isnull()]
full = full[full['time-event']!=2016]

In [144]:
full['event'].value_counts()

event
0.0    25931
1.0      771
Name: count, dtype: int64

In [135]:
full['Clasificación Industrial Internacional Uniforme Versión 4 A.C'] = full['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].apply(lambda x : x[0:1])

In [136]:
"""
actividades = {
    'A': 'Agricultura, ganadería, caza, silvicultura y pesca',
    'B': 'Explotación de minas y canteras',
    'C': 'Industrias manufacturetas',
    'D': 'Suministro de electricidad, gas, vapor y aire acondicionado',
    'E': 'Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneanmiento ambiental',
    'F': 'Construcción',
    'G': 'Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas',
    'H': 'Transporte y almacenamiento',
    'I': 'Alojamiento y servicios de comida',
    'J': 'Información y comunicaciones',
    'K': 'Actividades financieras y de seguros',
    'L': 'Actividades inmobiliarias',
    'M': 'Actividades profesionales,  científicas y técnicas',
    'N': 'Actividades de servicios administrativos y de apoyo',
    'O': 'Administración pública y defensa; planes de seguridad social de afiliación obligatoria',
    'P' :'Educación',
    'Q': 'Actividades de atención de la salud humana y de asistencia social',
    'R': 'Actividades artísticas, de entretenimiento y recreación',
    'S': 'Otras actividades de servicios',
    'T': 'Actividades de los hogares individuales en calidad de empleadores',
    'U': 'Actividades de organizaciones y entidades extraterritoriales'
}
full['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].replace(actividades, inplace=True)
"""

"\nactividades = {\n    'A': 'Agricultura, ganadería, caza, silvicultura y pesca',\n    'B': 'Explotación de minas y canteras',\n    'C': 'Industrias manufacturetas',\n    'D': 'Suministro de electricidad, gas, vapor y aire acondicionado',\n    'E': 'Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneanmiento ambiental',\n    'F': 'Construcción',\n    'G': 'Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas',\n    'H': 'Transporte y almacenamiento',\n    'I': 'Alojamiento y servicios de comida',\n    'J': 'Información y comunicaciones',\n    'K': 'Actividades financieras y de seguros',\n    'L': 'Actividades inmobiliarias',\n    'M': 'Actividades profesionales,  científicas y técnicas',\n    'N': 'Actividades de servicios administrativos y de apoyo',\n    'O': 'Administración pública y defensa; planes de seguridad social de afiliación obligatoria',\n    'P' :'Educación',\n    'Q': 'Actividad

In [137]:
full.to_csv("database-to-model.csv")

In [138]:
import numpy as np
import pandas as pd
df  =pd.DataFrame({'var1': [np.nan, 1, 2, 3],
'var2': [np.nan, np.nan,3,4],
'var3': [np.nan, 2, 3, np.nan]})

In [139]:
df.mean(axis=1, skipna=True)

0         NaN
1    1.500000
2    2.666667
3    3.500000
dtype: float64

In [140]:
df.mean(axis=1, skipna=False)

0         NaN
1         NaN
2    2.666667
3         NaN
dtype: float64

In [141]:
print((3+4)/2)

3.5


In [142]:
df.sum(axis=1)

0    0.0
1    3.0
2    8.0
3    7.0
dtype: float64

In [143]:
df.sum(axis=1, min_count=1)

0    NaN
1    3.0
2    8.0
3    7.0
dtype: float64